# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s):
###### Python version:
###### Submission deadline: 1pm, Fri 5 Apr 2019

In [290]:
# This function should open a data file in csv, and transform it into a usable format
import csv
def preprocess(filename, header):
    dataset = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile, header)
        for row in reader:
            dataset.append(row)
    return dataset



In [291]:
def train(dataset, header, classlabels):
    model = []
    smoothing = []
    classes = {}
    count = 0
    for index in range(len(dataset)):
        count += 1;
        if dataset[index]['class'] in classes:
            classes[dataset[index]['class']] += 1
        else:
            classes[dataset[index]['class']] = 1
    classes['count'] = count
    model.append(classes)
    
    #Laplace smoothing
    j = 0
    while j < len(header)-1:
        attributes = {}
        count = 0
        for index in range(len(dataset)):
            if dataset[index][header[j]] not in attributes:
                count += 1
                attributes[dataset[index][header[j]]] = 1
        j+=1
        attributes['count'] = count
        smoothing.append(attributes)
    i = 0
    
    while i < len(classlabels):
        current_class = classlabels[i]
        j = 0
        class_attributes = []
        while j < len(header)-1:
            attributes = smoothing[j].copy()
            for index in range(len(dataset)):
                if dataset[index]['class'] == current_class:
                    attributes['count'] += 1
                    attributes[dataset[index].get(header[j])] += 1
            class_attributes.append(attributes)
            j+=1
        model.append(class_attributes)
        i+=1
    return model

In [292]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, dataset, classlabels, header):
    predicted_classes = []
    highest_probability = 0
    probability = 0
    for index in range(len(dataset)):
        row = dataset[index]
        highest_probability = 0
        probability = 0
        predicted_label = ''
        for label in classlabels:
            probability = model[0][label]/model[0]['count']
            for attribute in range(len(row)-1):
                probability *= model[classlabels.index(label)+1][attribute][row.get(header[attribute])]/model[classlabels.index(label)+1][attribute]['count']
            if probability > highest_probability:
                highest_probability = probability
                predicted_label = label
        predicted_classes.append(predicted_label)
    return predicted_classes

In [293]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(predicted_classes, dataset):
    correct = 0
    wrong = 0
    count = 0
    for index in range(len(dataset)):
        row = dataset[index]
        if predicted_classes[index] == row['class']:
            correct += 1
        else:
            wrong += 1
        count += 1
    print("Accuracy is", correct/count*100,"% with", correct,"correct predictions and", wrong, "wrong predictions" )


In [345]:

import math
def info_gain(dataset,classlabels,headers):
    ig = {} #dict for information gain
    HR = 0 #total HR
    M = 0  # for mean infor
    V =[] 
    W=[] #store all the values in a list of a list
    
    #Get HR
    for j in range(len(classlabels)):
        HR += -((model[0][classlabels[j]]/model[0]['count'])*math.log2(model[0][classlabels[j]]/model[0]['count']))
    print(HR)
    
    #put all the value to a list W
    for k in range(len(header)-1):
        total_value = 0
        #put all the values into a list
        for i in range(len(model)-1):
            values = list(model[i+1][k].values())
            V.append(values)
        W.append(V)
        V=[]
        
    #Get the mean info
    M = att_em(W)
    
    #get final ig for each attributes
    for j in range(len(headers)-1):
        ig[headers[j]] = HR - M[j]
        print('HR:',HR,'M:',M[j])
    return ig

info_gain(dataset,classlabels,header)

1.7164959001837932
[[[1441, 1441, 1441, 4323], [3, 1, 1, 5], [197, 133, 1, 331], [1925, 1485, 859, 4269], [759, 1265, 2023, 4047]], [[865, 865, 865, 865, 865, 4325], [3, 1, 1, 1, 1, 7], [131, 133, 67, 1, 1, 333], [1345, 1345, 905, 465, 211, 4271], [253, 253, 759, 1265, 1519, 4049]], [[1081, 1081, 1081, 1081, 4324], [3, 1, 1, 1, 6], [119, 101, 71, 41, 332], [1153, 1093, 1039, 985, 4270], [889, 969, 1053, 1137, 4048]], [[1081, 1081, 1081, 1081, 4324], [3, 1, 1, 1, 6], [149, 101, 41, 41, 332], [1207, 1093, 985, 985, 4270], [805, 969, 1137, 1137, 4048]], [[1441, 1441, 1441, 4323], [3, 1, 1, 5], [209, 101, 21, 331], [1619, 1397, 1253, 4269], [1053, 1385, 1609, 4047]], [[2161, 2161, 4322], [3, 1, 4], [219, 111, 330], [2245, 2023, 4268], [1857, 2189, 4046]], [[1441, 1441, 1441, 4323], [2, 2, 1, 5], [165, 165, 1, 331], [1516, 1516, 1237, 4269], [1201, 1201, 1645, 4047]], [[1, 1, 4321, 4323], [3, 1, 1, 5], [329, 1, 1, 331], [2413, 1855, 1, 4269], [1579, 2467, 1, 4047]]]
p: 0.33397758020873597 H

{'parents': 0.06975802982107249,
 'has_nurs': 0.19468792866923312,
 'form': 0.0029281971117132155,
 'children': 0.009251795511775462,
 'housing': 0.016322369291984495,
 'finance': 0.0003605915886371047,
 'social': 0.02082535358608273,
 'health': 0.9573104824264627}

In [344]:
def att_em(values):
    print(values)
    M = 0 #for mean info
    mean = [] #list for all the mean info
    for i in range(len(values)):
        P = [] #for the prob
        H = [] #for the entropy
        k = 0 #count
        
        #iterate for each header
        for j in range(len(values[i])):
            while k < (len(values[i][j])-1):
                last_count = 0 #for P
                total_value=0 #total value of same attribute with different class
                for z in range(len(values[i])):
                    total_value+= values[i][z][k]
                    last_count += values[i][z][-1]
                H_ind = 0
                #get entropy for each
                for z in range(len(values[i])):
                    if ((values[i][z][k]-1)>1):
                        H_ind += -(((values[i][z][k]-1)/(total_value-len(values[i]))))*\
                        (math.log2((values[i][z][k]-1)/(total_value-len(values[i]))))
                H.append(H_ind)
                P.append((total_value-len(values[i]))/(last_count-(len(values[i])*len(values))))
                k+=1
        M = 0
        for z in range(len(H)):
            M += P[z]*H[z]
            print('p:',P[z],'H',H[z])
        mean.append(M)
    return mean
                
        
        
        

In [383]:
def hold_out(dataset):
    train = 0
    test = 0
    print(len(dataset))
    if (len(dataset)%2 == 0):
        train = dataset[0:(len(dataset)//2)]
        test  = dataset[(len(dataset)//2):len(dataset)]
        print(len(train),len(test))
    else:
        train = dataset[0:((len(dataset)-1)//2)-1]
        test  = dataset[(len(dataset)-1//2):len(dataset)]
        
    return train, test
hold_out(dataset)
    
    

12960
6480 6480


([OrderedDict([('parents', 'usual'),
               ('has_nurs', 'proper'),
               ('form', 'complete'),
               ('children', '1'),
               ('housing', 'convenient'),
               ('finance', 'convenient'),
               ('social', 'nonprob'),
               ('health', 'recommended'),
               ('class', 'recommend')]),
  OrderedDict([('parents', 'usual'),
               ('has_nurs', 'proper'),
               ('form', 'complete'),
               ('children', '1'),
               ('housing', 'convenient'),
               ('finance', 'convenient'),
               ('social', 'nonprob'),
               ('health', 'priority'),
               ('class', 'priority')]),
  OrderedDict([('parents', 'usual'),
               ('has_nurs', 'proper'),
               ('form', 'complete'),
               ('children', '1'),
               ('housing', 'convenient'),
               ('finance', 'convenient'),
               ('social', 'nonprob'),
               ('health', 'not_

In [175]:
filename = 'nursery.csv'
header = 'parents','has_nurs','form','children','housing','finance','social','health','class'
classlabels = 'not_recom', 'recommend', 'very_recom', 'priority', 'spec_prior'
dataset = preprocess(filename, header)
model = train(dataset, header, classlabels)
predicted_classes = predict(model,dataset,classlabels, header)
evaluate(predicted_classes, dataset)


Accuracy is 90.30092592592592 % with 11703 correct predictions and 1257 wrong predictions


Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

1.Yes, when the information gain of each attribute is higher than others, then it means that it can be a good choice.


MISS VALUE
1.can take question mark as a value which the attribute will have 3 attributes which are T,F,? just treat it as a single value
2.still have question marks in the set but - the question marks at the end. 1/3-1
3.it is not nessary to 
Question 2: 
evidence of independance  calculate the attribute one and two's information gain. so not calculating the class but the infomation gain between two attributes

for all the attributes get the information gain and put them in a list
information gain:
just culculate 4 of them.


4,5(add_k)
